# Object Detection

## Using yolo algo 

In [6]:
import cv2
import numpy as np
yolo_net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
layer_names=yolo_net.getLayerNames()
output_layers=[layer_names[i-1] for i in yolo_net.getUnconnectedOutLayers()]
with open("coco.names","r") as f:
    classes=[line.strip() for line in f.readlines()]

## Step 1: Load and preprocess the image

In [7]:
def preprocess_image(image_path):
    image=cv2.imread(image_path)
    height,width,channels=image.shape
    blob=cv2.dnn.blobFromImage(image,0.00392,(416,416),(0,0,0),True,crop=False) 
    yolo_net.setInput(blob) 
    return image,height,width

## Step 2: Detect objects using YOLO

In [8]:
def detect_objects(image_path):
    image,height,width=preprocess_image(image_path)    
    outputs=yolo_net.forward(output_layers)
    class_ids=[]
    confidences=[]
    boxes=[]
    for output in outputs:
        for detection in output:
            scores=detection[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:
                center_x=int(detection[0]*width)
                center_y=int(detection[1]*height)
                w=int(detection[2]*width)
                h=int(detection[3]*height)
                x=int(center_x-w/2)
                y=int(center_y-h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indices=cv2.dnn.NMSBoxes(boxes,confidences,score_threshold=0.5,nms_threshold=0.4)
    return image,boxes,confidences,class_ids,indices

## Step 3: Draw bounding boxes and labels on the image

In [9]:
def draw_labels(image,boxes,confidences,class_ids,indices):
    for i in indices.flatten():
        x,y,w,h=boxes[i]
        label=str(classes[class_ids[i]])
        confidence=str(round(confidences[i],2))      
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(image,label+" "+confidence,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
    return image

# Main code

In [10]:
image_path='pic.jpeg'
image,boxes,confidences,class_ids,indices=detect_objects(image_path)
image_with_labels=draw_labels(image,boxes,confidences,class_ids,indices)
cv2.imshow("Detected Objects",image_with_labels)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'